xmk: 我们尝试一下自己做，最终的目的就是要获取一下embedding。

In [4]:
import matplotlib.pyplot as plt
# from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
# from stellargraph.data import EdgeSplitter
# from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

%matplotlib inline

/Users/minkexiu/opt/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
dataset = datasets.Cora()
display(HTML(dataset.description))
graph, _ = dataset.load(largest_connected_component_only=True, str_node_ids=True)

In [3]:
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 2485, Edges: 5209

 Node types:
  paper: [2485]
    Features: float32 vector, length 1433
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5209]
        Weights: all 1 (default)
        Features: none


In [4]:
from stellargraph.data import BiasedRandomWalk


def create_biased_random_walker(graph, walk_num, walk_length):
    # parameter settings for "p" and "q":
    p = 1.0
    q = 1.0
    return BiasedRandomWalk(graph, n=walk_num, length=walk_length, p=p, q=q)

In [5]:
walk_length = 5
epochs = 6
batch_size = 50

In [6]:
from stellargraph.data import UnsupervisedSampler
from tensorflow import keras
from stellargraph.layer import link_classification

In [7]:
from stellargraph.mapper import GraphSAGELinkGenerator, GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

# def graphsage_embedding(graph, name):

# Set the embedding dimensions, the numbers of sampled neighboring nodes and walk number:
dimensions = [128, 128]
num_samples = [10, 5]
walk_number = 1

# print(f"Training GraphSAGE for '{name}':")

graph_node_list = list(graph.nodes())

# Create the biased random walker to generate random walks
walker = create_biased_random_walker(graph, walk_number, walk_length)

# Create the unsupervised sampler to sample (target, context) pairs from random walks
unsupervised_samples = UnsupervisedSampler(
    graph, nodes=graph_node_list, walker=walker
)

# Define a GraphSAGE training generator, which generates batches of training pairs
generator = GraphSAGELinkGenerator(graph, batch_size, num_samples)

# Create the GraphSAGE model
graphsage = GraphSAGE(
    layer_sizes=dimensions,
    generator=generator,
    bias=True,
    dropout=0.0,
    normalize="l2",
)

# Build the model and expose input and output sockets of GraphSAGE, for node pair inputs
x_inp, x_out = graphsage.in_out_tensors()

# Use the link_classification function to generate the output of the GraphSAGE model
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

# Stack the GraphSAGE encoder and prediction layer into a Keras model, and specify the loss
model = keras.Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

# Train the model
model.fit(
    generator.flow(unsupervised_samples),
    epochs=epochs,
    verbose=2,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

# Build the model to predict node representations from node features with the learned GraphSAGE model parameters
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

# Get representations for all nodes in ``graph``
node_gen = GraphSAGENodeGenerator(graph, batch_size, num_samples).flow(
    graph_node_list
)
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

# def get_embedding(u):
#     u_index = graph_node_list.index(u)
#     return node_embeddings[u_index]

# return get_embedding

link_classification: using 'ip' method to combine node embeddings into edge embeddings


/Users/minkexiu/opt/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2023-11-23 17:19:29.942020: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/6
398/398 - 8s - loss: 0.5600 - binary_accuracy: 0.7515 - 8s/epoch - 19ms/step
Epoch 2/6
398/398 - 7s - loss: 0.5411 - binary_accuracy: 0.7743 - 7s/epoch - 18ms/step
Epoch 3/6
398/398 - 7s - loss: 0.5384 - binary_accuracy: 0.7847 - 7s/epoch - 18ms/step
Epoch 4/6
398/398 - 7s - loss: 0.5346 - binary_accuracy: 0.7909 - 7s/epoch - 19ms/step
Epoch 5/6
398/398 - 7s - loss: 0.5349 - binary_accuracy: 0.7944 - 7s/epoch - 18ms/step
Epoch 6/6
398/398 - 7s - loss: 0.5362 - binary_accuracy: 0.7930 - 7s/epoch - 18ms/step


In [9]:
node_embeddings.shape

(2485, 128)

In [12]:
pd.DataFrame(node_embeddings, columns = [f"col_{i}" for i in range(node_embeddings.shape[1])])

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_118,col_119,col_120,col_121,col_122,col_123,col_124,col_125,col_126,col_127
0,0.125639,0.037899,-0.060162,-0.034127,0.039009,-0.092451,-0.040883,-0.143016,-0.036907,-0.110292,...,-0.005976,0.100427,0.139106,-0.024013,0.121216,0.038344,-0.081775,0.243015,-0.122498,0.105455
1,-0.037125,-0.084273,-0.025988,-0.007329,-0.035599,-0.005473,0.048691,0.027475,0.032161,0.038053,...,-0.102702,-0.276421,0.096836,-0.121819,-0.215990,-0.078925,-0.135694,0.039910,0.188367,0.046014
2,-0.007121,-0.013519,0.060606,0.008421,0.016279,0.017287,-0.050240,-0.043012,0.052959,0.010345,...,0.044987,-0.007048,-0.057462,-0.034908,0.133628,0.071029,0.000228,-0.007571,0.093483,-0.137894
3,-0.026033,0.013021,0.039247,0.020212,0.016811,0.000646,-0.057095,-0.037960,0.025462,-0.025608,...,0.056685,-0.000662,-0.081845,-0.025931,0.129672,0.094335,0.012142,0.006770,0.106190,-0.149273
4,-0.002963,-0.039001,0.149685,-0.141665,0.065819,-0.096245,0.000559,-0.130199,-0.035878,-0.040521,...,0.004715,0.134088,-0.046262,0.027129,0.174941,0.060874,0.083064,0.080971,-0.022242,0.079496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,0.037329,0.048570,0.019255,-0.009525,-0.027434,0.015536,0.051639,-0.031082,-0.016124,0.011653,...,0.169767,0.143130,0.169336,0.268570,0.192503,0.004831,-0.047785,0.027546,-0.183254,0.007093
2481,-0.005255,0.088725,-0.021719,0.021321,-0.044354,0.016525,0.044843,-0.020543,-0.019718,0.048924,...,0.183138,0.120295,0.149323,0.259667,0.170614,-0.014542,-0.041933,0.046331,-0.129212,-0.027432
2482,0.009050,0.082403,0.003552,-0.003589,-0.052748,0.057156,0.078848,-0.031178,0.002797,0.055609,...,0.152775,0.104083,0.163879,0.256179,0.134577,-0.021883,-0.051193,0.033391,-0.180290,0.014722
2483,-0.159819,0.019725,0.009750,0.013587,-0.022355,-0.028463,-0.031145,0.019994,-0.109458,-0.073868,...,0.114370,-0.050271,-0.111632,0.119926,-0.045934,0.098546,0.010439,0.033512,0.178907,-0.175016


In [1]:
pwd

'/Users/minkexiu/Documents/GitHub/ML_Tryout/Graph_Series'

In [2]:
!ls ~

2.0.0                           Public
Applications                    df_fdt.csv
Desktop                         java_error_in_pycharm_22319.log
Documents                       java_error_in_pycharm_5789.log
Downloads                       java_error_in_pycharm_74657.log
IdeaProjects                    jbr_err_pid74657.log
Library                         nltk_data
LightGBM_compilation.log        opt
Movies                          sn-article.tex
Music                           stellargraph-datasets
OneDrive                        total_20220401_20220915.csv
Pictures                        xyz.java
Postman                         同步空间


# 自己设计数据集

https://stellargraph.readthedocs.io/en/stable/demos/basics/loading-pandas.html

我们可以构建一个带边权重的有向图。

然后我们再构建一个特征集，特征集的 index 要是 companyid，或者是什么注册号之类的。。

In [13]:
square_edges = pd.DataFrame(
    {"source": ["a", "b", "c", "d", "a"], "target": ["b", "c", "d", "a", "c"]}
)
square_edges

,source,target
0,a,b
1,b,c
2,c,d
3,d,a
4,a,c


In [14]:
square = StellarGraph(edges=square_edges)

In [15]:
square.nodes()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [20]:
graph.node_features().shape

(2485, 1433)